# ***Distribución ESTACIONAMIENTOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales     
    if nombre.startswith("df_")  and "puebla" in nombre and 'df_alfa_q_puebla' not in nombre and 'financiamientos' not in nombre and 'grupos' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre

]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Estacionamiento':'estacionamiento','Estacionamientos':'estacionamiento','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','estacionamiento','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [5]:
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [6]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [1, 2, 3, 4, 5, 6, float('inf')]
        labels = ['1', '2', '3', '4', '5', '6']
        df['estacionamiento'] = pd.cut(df['estacionamiento'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'estacionamiento']).size()

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3394540407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3394540407.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3394540407.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Nombre del DataFrame: df_alfa_agosto_2024_puebla
segmento  estacionamiento
A         1                   38
          2                  135
          3                   40
          4                   57
          5                    0
          6                    0
B         1                   42
          2                  159
          3                   58
          4                    0
          5                    0
          6                    0
C         1                   66
          2                  133
          3                  161
          4                    0
          5                    0
          6                    0
D         1                  114
          2                  147
          3                  151
          4                    0
          5                    0
          6                    0
E         1                  139
          2                  888
          3                    2
          4                   15
 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3394540407.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['estacionamiento'] = pd.cut(df['estacionamiento'], bins=bins, labels=labels, right=False)


In [7]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'estacionamiento'],observed=False).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
estacionamiento       1       2       3       4      5      6
segmento                                                     
A                1004.0  1924.0   281.0   282.0   20.0   19.0
B                1147.0  2224.0   365.0    53.0    6.0    8.0
C                1478.0  1917.0   864.0    11.0    1.0   12.0
D                1770.0  1753.0   688.0     6.0    1.0    0.0
E                1919.0  4554.0    32.0    55.0   29.0  136.0
L                 233.0   232.0    75.0    96.0  157.0  401.0
S                1437.0  2248.0  1169.0  1066.0  323.0  554.0


In [8]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
         # Aplica el orden a la columna 'segmento'
        df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
        
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
         yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        legend_title="Estacionamientos",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3575200082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3575200082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10472\3575200082.py:9: SettingWithCopyWarning: 
A value is try

Tabla general de frecuencia:
segmento            E     D     C     B     A     S    L
estacionamiento                                         
1                1919  1770  1478  1147  1004  1437  233
2                4554  1753  1917  2224  1924  2248  232
3                  32   688   864   365   281  1169   75
4                  55     6    11    53   282  1066   96
5                  29     1     1     6    20   323  157
6                 136     0    12     8    19   554  401


In [9]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#F79CB9', '#E50CB6', '#4225CF']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    # Sumar los valores para cada estacionamiento
    suma_estacionamientos = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_estacionamientos.index,
        values=suma_estacionamientos.values,
        marker=dict(colors=colores[:len(suma_estacionamientos)]),
        #hole=.3  # Si deseas un gráfico de pastel tipo "donut"
    )])
    
    fig.update_layout(
        #title="Distribución de Estacionamientos por Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    )
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento            E     D     C     B     A     S    L
estacionamiento                                         
1                1919  1770  1478  1147  1004  1437  233
2                4554  1753  1917  2224  1924  2248  232
3                  32   688   864   365   281  1169   75
4                  55     6    11    53   282  1066   96
5                  29     1     1     6    20   323  157
6                 136     0    12     8    19   554  401
